In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

db=[]
for i in range(0,7):
    #img1 = cv.imread('db/juice_train0.jpg',0)
    img1 = cv2.imread('db/img'+str(i)+'.jpg',0)
    # Initiate ORB detector
    orb = cv2.ORB_create()
    # find the keypoints with ORB
    kp1 = orb.detect(img1,None)

    # compute the descriptors with ORB
    kp1, des1 = orb.compute(img1, kp1)
    db.append([img1,kp1,des1])


In [ ]:
#!/usr/bin/env python

import sys
import cv2
 
if __name__ == '__main__':
    # If image path and f/q is not passed as command
    # line arguments, quit and display help message
     
    # speed-up using multithreads
    cv2.setUseOptimized(True);
    cv2.setNumThreads(4);
 
    # read image
    for i in range(0,15):
        im = cv2.imread('db2/img'+str(i)+'.jpg',1)
        cv2.imshow("Output", im)

        # key press
        cv2.waitKey(0)

        #im = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # resize image
        newHeight = 200
        newWidth = int(im.shape[1]*200/im.shape[0])
        im = cv2.resize(im, (newWidth, newHeight))    

        # create Selective Search Segmentation Object using default parameters
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

        # set input image on which we will run segmentation
        ss.setBaseImage(im)

        # Switch to fast but low recall Selective Search method
        #ss.switchToSelectiveSearchFast()

        # Switch to high recall but slow Selective Search method
        ss.switchToSelectiveSearchQuality()
        # if argument is neither f nor q print help message

        # run selective search segmentation on input image
        rects = ss.process()
        print('Total Number of Region Proposals: {}'.format(len(rects)))

        # number of region proposals to show
        numShowRects = 5
        # increment to increase/decrease total number
        # of reason proposals to be shown
        increment = 10

        # create a copy of original image
        imOut = im.copy()

        # itereate over all the region proposals
        x_mean=0
        y_mean=0
        for i, rect in enumerate(rects):
            # draw rectangle for region proposal till numShowRects
            if (i < numShowRects):
                x, y, w, h = rect
                cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
                x_mean+=x
                y_mean+=y
            else:
                break
        x_mean=x_mean/numShowRects
        y_mean=y_mean/numShowRects
        
        cv2.rectangle(imOut, (int(x_mean), int(y_mean)), (int(x_mean)+100, int(y_mean)+100), (255, 255, 0), 1, cv2.LINE_AA)
        # show output
        cv2.imshow("Output", imOut)
        cv2.waitKey()

    # record key press
    k = cv2.waitKey(0) & 0xFF

    # m is pressed
    if k == 109:
        # increase total number of rectangles to show by increment
        numShowRects += increment
    # l is pressed
    elif k == 108 and numShowRects > increment:
        # decrease total number of rectangles to show by increment
        numShowRects -= increment
    # q is pressed
    elif k == 113:
        print('break')
    # close image show window
    cv2.destroyAllWindows()

Total Number of Region Proposals: 769
Total Number of Region Proposals: 1117
Total Number of Region Proposals: 1163
Total Number of Region Proposals: 1258
Total Number of Region Proposals: 1335
Total Number of Region Proposals: 1170
Total Number of Region Proposals: 671
Total Number of Region Proposals: 681
Total Number of Region Proposals: 771
Total Number of Region Proposals: 954
Total Number of Region Proposals: 974
Total Number of Region Proposals: 1379
Total Number of Region Proposals: 1387


In [ ]:
import cv2
import sys
 
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')￼
 
if __name__ == '__main__' :
 
    # Set up tracker.
    # Instead of MIL, you can also use
 
    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    tracker_type = tracker_types[2]
 
    if int(minor_ver) < 3:
        tracker = cv2.Tracker_create(tracker_type)
    else:
        if tracker_type == 'BOOSTING':
            tracker = cv2.TrackerBoosting_create()
        if tracker_type == 'MIL':
            tracker = cv2.TrackerMIL_create()
        if tracker_type == 'KCF':
            tracker = cv2.TrackerKCF_create()
        if tracker_type == 'TLD':
            tracker = cv2.TrackerTLD_create()
        if tracker_type == 'MEDIANFLOW':
            tracker = cv2.TrackerMedianFlow_create()
        if tracker_type == 'GOTURN':
            tracker = cv2.TrackerGOTURN_create()
        if tracker_type == 'MOSSE':
            tracker = cv2.TrackerMOSSE_create()
        if tracker_type == "CSRT":
            tracker = cv2.TrackerCSRT_create()
 
    # Read video
    video = cv2.VideoCapture("videos/chaplin.mp4")
 
    # Exit if video not opened.
    if not video.isOpened():
        print "Could not open video"
        sys.exit()
 
    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print 'Cannot read video file'
        sys.exit()
     
    # Define an initial bounding box
    bbox = (287, 23, 86, 320)
 
    # Uncomment the line below to select a different bounding box
    bbox = cv2.selectROI(frame, False)
 
    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)
 
    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break
         
        # Start timer
        timer = cv2.getTickCount()
 
        # Update tracker
        ok, bbox = tracker.update(frame)
 
        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
 
        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
 
        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
     
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
 
        # Display result
        cv2.imshow("Tracking", frame)
 
        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : break